In [1]:
'''
In order to properly setup my local environment:
- pip install boto3
- I had to install Spark, Java, Hadoop, and SBT on my Windows 10 machine, ADD YT LINK FOR SBT TOO!!
following the instructions at this link: https://www.youtube.com/watch?v=g7Qpnmi0Q-s for the Java/Hadoop portion.
'''

'\nIn order to properly setup my local environment:\n- pip install boto3\n- I had to install Spark, Java, Hadoop, and SBT on my Windows 10 machine, ADD YT LINK FOR SBT TOO!!\nfollowing the instructions at this link: https://www.youtube.com/watch?v=g7Qpnmi0Q-s for the Java/Hadoop portion.\n'

In [72]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql import types as T
import  pyspark.sql.functions as F
import time
from pyspark.sql.functions import monotonically_increasing_id

In [3]:
config = configparser.ConfigParser()
config.read('dl.cfg')

['dl.cfg']

In [4]:
os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [5]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [6]:
spark = create_spark_session()
spark

In [7]:
# use this to speed up parquet write
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("mapreduce.fileoutputcommitter.algorithm.version", "2")

In [8]:
sc._jsc.hadoopConfiguration().set("spark.speculation","false")

In [9]:
# Not sure that second line is faster than first
#df = spark.read.format("json").load("s3a://udacity-dend/song_data/*/*/*") # runs in 13 minutes
#df = spark.read.format("json").load(file_locations)
# df = spark.read.format("json").load(songs_paths)
# song_data = input_data+'song_data/*/*/*/*.json
# song_data = input_data + 'song_data/*/*/*/*.json # runs in 14 minutes
# df = spark.read.json(song_data) # runs in 14 minutes
# df = spark.read.json(songs_paths) # runs in 26 minutes

start = time.time()
#input_data = "s3a://udacity-dend/"
#song_data =  input_data + 'song_data/*/*/*/*.json'
df_song_data = spark.read.json("s3a://udacity-dend/song_data/*/*/*") # runs in 13 minutes
end = time.time()
print('runtime (s):', end-start)

runtime (s): 492.0592863559723


In [10]:
'''
start = time.time()
print(type(df))
print(df)
print('count', df.count())
#df.printSchema()
end = time.time()
print('runtime (s):', end-start)
'''

"\nstart = time.time()\nprint(type(df))\nprint(df)\nprint('count', df.count())\n#df.printSchema()\nend = time.time()\nprint('runtime (s):', end-start)\n"

In [11]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://dend-sparkify-amiri/"

In [14]:
# extract columns to create songs table
start = time.time()
songs_table = df_song_data.select("song_id", "title", "artist_id", "year", "duration").distinct()
end = time.time()
print('extract columns to create songs table runtime (s):', end-start)

# write songs table to parquet files partitioned by year and artist (tried using alias for artist_name, partitionBy does not seem to like this)
start = time.time()
songs_table_parquet = df_song_data.select("song_id", "title", "artist_id", "year", "duration", "artist_name").distinct()
#songs_table_parquet.write.mode('overwrite').partitionBy("year", "artist").parquet(output_data + "songs/") # S3 location
songs_table_parquet.write.mode('overwrite').partitionBy("year", "artist_id").parquet('output_data/' + "songs/") # local location
end = time.time()
print('write songs table to parquet files runtime (s):', end-start)

extract columns to create songs table runtime (s): 0.03291201591491699
write songs table to parquet files runtime (s): 389.07194328308105


In [13]:
# extract columns to create artists table
start = time.time()
artists_table = df_song_data.select("artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude").distinct()
end = time.time()
print('extract columns to create artists table runtime (s):', end-start)

# write artists table to parquet files
start = time.time()
artists_table_parquet = artists_table.write.mode('overwrite').parquet(output_data + "artists/")
end = time.time()
print('write artists table to parquet files runtime (s):', end-start)

extract columns to create artists table runtime (s): 0.029893875122070312
write artists table to parquet files runtime (s): 409.5495846271515


## Put finalized code from above in sections below, then transfer to .py file. 

In [ ]:
def process_song_data(spark, input_data, output_data):
    '''
    This function reads the data from S3, processes the song data using Spark, and writes the processed data back to S3.

    Parameters:
    - spark: The spark session
    - input_data: The S3 path location up to, but not including `song_data`
    - output_data: The S3 bucket where the new dimensional tables will be written to
    '''
    # get filepath to song data file
    start = time.time()
    song_data =  input_data + 'song_data/*/*/*/*.json'
    end = time.time()
    print('runtime (s):', end-start)
    
    # read song data file
    start = time.time()
    df = spark.read.json(song_data) # may not need this new schema
    end = time.time()
    print('read song data file runtime (s):', end-start)

    # extract columns to create songs table
    start = time.time()
    songs_table = df.select("song_id", "title", "artist_id", "year", "duration").distinct()
    end = time.time()
    print('extract columns to create songs table runtime (s):', end-start)

    # write songs table to parquet files partitioned by year and artist
    start = time.time()
    songs_table_parquet = df.select("song_id", "title", "artist_id", "year", "duration", col("artist_name").alias("artist")).distinct()
    songs_table_parquet.write.mode('overwrite').partitionBy("year", "artist_id").parquet(output_data + "songs/")
    end = time.time()
    print('write songs table to parquet files runtime (s):', end-start)

    # extract columns to create artists table
    start = time.time()
    artists_table = df.select("artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude").distinct()
    end = time.time()
    print('extract columns to create artists table runtime (s):', end-start)

    # write artists table to parquet files
    start = time.time()
    artists_table_parquet = artists_table.write.mode('overwrite').parquet(output_data + "artists/")
    end = time.time()
    print('write artists table to parquet files runtime (s):', end-start)

In [ ]:
def process_log_data(spark, input_data, output_data):
   '''
    This function reads the data from S3, processes the log data using Spark,
    and writes the processed data back to S3.
   '''
    # get filepath to log data file
    start = time.time()
    log_data = input_data + 'log_data/*/*/*.json'
    end = time.time()
    print('get log filepath runtime (s):', end-start)

    # read log data file
    start = time.time()
    df = spark.read.json(log_data)
    end = time.time()
    print('read log data file runtime (s):', end-start)

    # filter by actions for song plays
    df = df.filter(df.page=='NextSong')

    # extract columns for users table    
    start = time.time()
    users_table = df.select("user_id", "first_name", "last_name", "gender", "level").distinct()
    end = time.time()
    print('extract columns for users table runtime (s):', end-start)
    
    # write users table to parquet files
    start = time.time()
    users_table.write.mode('overwrite').partitionBy("year", "artist_id").parquet(output_data + "users/")
    end = time.time()
    print('write users table to parquet files runtime (s):', end-start)

    # create timestamp column from original timestamp column
    get_timestamp = F.udf(lambda x: datetime.fromtimestamp(x/1000.0), T.TimestampType() )
    df = df.withColumn("timestamp", get_timestamp(df.ts)) # creates new column named timestamp, populates it with converted timestamp
    
    # create datetime column from original timestamp column
    get_datetime = F.udf(lambda x: datetime.fromtimestamp(x/1000.0) )
    df = df.withColumn("datetime", get_timestamp(df.ts))
    
    # extract columns to create time table
    time_table = df.select(datetime)
    time_table = time_table.withColumn("month", F.month("datetime"), )
    
    # write time table to parquet files partitioned by year and month
    time_table.write.mode('overwrite').partitionBy("year", "month").parquet(output_data + "time/")

    # read in song data to use for songplays table
    song_df = 

    # extract columns from joined song and log datasets to create songplays table 
    songplays_table = 

    # write songplays table to parquet files partitioned by year and month
    songplays_table

In [ ]:
def main():
    '''
    Create Spark session, provide paths to input/output data, load songs/log data
    and create parquet tables (columnar format) with star schema DB.
    '''
    spark = create_spark_session()
    input_data = "s3a://udacity-dend/"
    output_data = "s3a://dend-sparkify-amiri/output_data"
    
    process_song_data(spark, input_data, output_data)    
    process_log_data(spark, input_data, output_data)

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
# SANDBOX AREA BELOW ############

In [15]:
# get filepath to log data file
start = time.time()
log_data = input_data + 'log_data/*/*/*.json'
end = time.time()
print('get log filepath runtime (s):', end-start)

get log filepath runtime (s): 0.0


In [16]:
# read log data file
start = time.time()
df_log_data = spark.read.json(log_data)
end = time.time()
print('read log data file runtime (s):', end-start)

read log data file runtime (s): 8.51508903503418


In [17]:
df_log_data

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: double, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [22]:
# filter by actions for song plays
df_log_data = df_log_data.filter(df_log_data.page=='NextSong')

# extract columns for users table    
start = time.time()
users_table = df_log_data.select("userid", "firstName", "lastName", "gender", "level").distinct()
end = time.time()
print('extract columns for users table runtime (s):', end-start)

extract columns for users table runtime (s): 0.017918825149536133


In [23]:
# write users table to parquet files
start = time.time()
users_table.write.mode('overwrite').parquet(output_data + "users/")
end = time.time()
print('write users table to parquet files runtime (s):', end-start)

write users table to parquet files runtime (s): 53.5797758102417


In [52]:
'''
# create timestamp column from original timestamp column
get_timestamp = F.udf(lambda x: datetime.fromtimestamp(x/1000.0), T.TimestampType() )
df = df.withColumn("timestamp", get_timestamp(df.ts)) # creates new column named timestamp, populates it with converted timestamp
'''

In [24]:
# create datetime column from original timestamp column
get_datetime = F.udf(lambda x: datetime.fromtimestamp(x/1000.0), T.TimestampType() )
df_log_data = df_log_data.withColumn("start_time", get_datetime(df_log_data.ts))

In [ ]:
# write users table to parquet files
users_table.write.mode('overwrite').parquet(output_data + "users/")

In [25]:
# extract columns to create time table
time_table = df_log_data.select("start_time").distinct().withColumn("hour",    F.hour(df_log_data.start_time)) \
                                                        .withColumn("day",     F.dayofmonth(df_log_data.start_time)) \
                                                        .withColumn("week",    F.weekofyear(df_log_data.start_time)) \
                                                        .withColumn("month",   F.month(df_log_data.start_time)) \
                                                        .withColumn("year",    F.year(df_log_data.start_time)) \
                                                        .withColumn("weekday", F.date_format(df_log_data.start_time, "E")) # the 'E' formats this parameter to day-of-the-week

In [47]:
time_table.show(3)

+--------------------+----+---+----+-----+----+-------+
|          start_time|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-20 23:18:...|  23| 20|  47|   11|2018|    Tue|
|2018-11-21 11:49:...|  11| 21|  47|   11|2018|    Wed|
|2018-11-14 08:20:...|   8| 14|  46|   11|2018|    Wed|
+--------------------+----+---+----+-----+----+-------+
only showing top 3 rows



In [28]:
# write time table to parquet files partitioned by year and month
time_table.write.mode('overwrite').partitionBy("year", "month").parquet(output_data + "time/") 

In [29]:
# read in song data to use for songplays table
song_df_path = 'output_data/' + 'songs/*/*/*'
#song_df_path = r'C:\Users\amiri\Documents\GitHub\Data_Engineering_Data_Lake_with_Spark\output_data'
#song_df = spark.read.parquetoutput_data + 'songs/*/*/*')
song_df = spark.read.parquet(song_df_path)

In [40]:
print(type(song_df))
song_df.show(3)

<class 'pyspark.sql.dataframe.DataFrame'>
+------------------+--------------------+---------+--------------------+
|           song_id|               title| duration|         artist_name|
+------------------+--------------------+---------+--------------------+
|SOEPTVC12A67ADD0DA|To Zucchabar ["Gl...|196.04853|Yvonne S. Moriart...|
|SOVPFJK12A6701CB16|Barcelona - (Frie...|273.44934|Russell Watson / ...|
|SOQEBML12A8C136AA4|Werther (1997 Dig...| 55.40526|Alfredo Kraus/Tat...|
+------------------+--------------------+---------+--------------------+
only showing top 3 rows



In [55]:
print(type(df_log_data))
df_log_data.show(3)

<class 'pyspark.sql.dataframe.DataFrame'>
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+--------------------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|           song|status|           ts|           userAgent|userId|          start_time|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+--------------------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|  Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|2018-11-14 17:30:...|
|The Prodigy|Logged In|     Ryan| 

In [46]:
song_df.printSchema()

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- artist_name: string (nullable = true)



In [45]:
#df_log_data.schema.names
df_log_data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: timestamp (nullable = true)



In [58]:
#df_log_data.schema.names
artists_df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)



In [ ]:
# extract columns from joined song and log datasets to create songplays table 
# songplays table columns: songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
# need to get artist_id

In [52]:
# read in artist data so that i have access to artist_id column
artists_df = spark.read.parquet(output_data + 'artists/*')

In [56]:
# join songs are log data on title column
songs_logs_df = df_log_data.join(song_df, df_log_data.song == song_df.title)

In [67]:
songs_artists_df = songs_logs_df.join(artists_df, songs_logs_df.artist_name == artists_df.artist_name)

In [68]:
songs_artists_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_loca

In [69]:
# songplays table columns: songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
songplays_table = songs_artists_df.select("start_time", "userId", "level", "song_id", "artist_id", "sessionId", "location", "userAgent").distinct()

In [79]:
songplays_table = songplays_table.withColumn("songplay_id", monotonically_increasing_id()) # create songplay_id and make it primary key
songplays_table = songplays_table.select("songplay_id", "start_time", "userId", "level", "song_id", "artist_id", "sessionId", "location", "userAgent") # rearrange
songplays_table = songplays_table.withColumn("year", F.year("start_time")).withColumn("month", F.month("start_time"))
songplays_table.show(3)

+-----------+--------------------+------+-----+------------------+------------------+---------+--------------------+--------------------+----+-----+
|songplay_id|          start_time|userId|level|           song_id|         artist_id|sessionId|            location|           userAgent|year|month|
+-----------+--------------------+------+-----+------------------+------------------+---------+--------------------+--------------------+----+-----+
|          0|2018-11-29 09:58:...|    49| paid|SOGXSWA12A6D4FBC99|ARPFHN61187FB575F6|     1041|San Francisco-Oak...|Mozilla/5.0 (Wind...|2018|   11|
|          1|2018-11-19 18:57:...|    85| paid|SORZRUJ12A6D4F61A6|ARWVYIM11F4C8411E4|      658|       Red Bluff, CA|"Mozilla/5.0 (Mac...|2018|   11|
|          2|2018-11-16 12:22:...|    49| paid|SOAHNFK12AC9075097|ARZIFFT1187FB41B2A|      648|San Francisco-Oak...|Mozilla/5.0 (Wind...|2018|   11|
+-----------+--------------------+------+-----+------------------+------------------+---------+-----------

In [81]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write.mode('overwrite').partitionBy("year", "month").parquet(output_data + "songplays/")

In [ ]:
'''
# This code is from the internet, and it works
from pyspark import SparkContext
sc = SparkContext(master = 'local')
create_spark_session()
'''

In [ ]:
# Works! Stops the spark context
SparkSession.stop()

In [ ]:
# !pip install boto3
# Also had to install Java SDK

In [ ]:
# Link to udacity bucket with the console:  https://s3.console.aws.amazon.com/s3/buckets/udacity-dend/?region=us-west-2
#s3path = 'https://s3.console.aws.amazon.com/s3/buckets/udacity-dend/?region=us-west-2#'
#s3pathsong = 'https://s3.console.aws.amazon.com/s3/buckets/udacity-dend/song_data/A/?region=us-west-2'
